In [ ]:
"C:\Users\10671\PycharmProjects\pythonProject\relax\visual rock.jpg"

In [9]:
import cv2
import mediapipe as mp
import random
import time
import numpy as np

def get_hand_sign(hand_landmarks):
    tip_ids = [4, 8, 12, 16, 20]
    fingers = []
    if hand_landmarks.landmark[tip_ids[0]].x < hand_landmarks.landmark[tip_ids[0]-1].x:
        fingers.append(1)
    else:
        fingers.append(0)
    for i in range(1,5):
        if hand_landmarks.landmark[tip_ids[i]].y < hand_landmarks.landmark[tip_ids[i]-2].y:
            fingers.append(1)
        else:
            fingers.append(0)
    if fingers == [0,0,0,0,0] or fingers == [1,0,0,0,0]:
        return 'Rock'
    elif fingers == [0,1,1,0,0] or fingers == [1,1,1,0,0]:
        return 'Scissors'
    elif fingers == [0,1,1,1,1] or fingers == [1,1,1,1,1]:
        return 'Paper'
    else:
        return 'what the fuck'

def decide_winner(user, comp):
    if user == comp:
        return 'Draw'
    elif (user == 'Rock' and comp == 'Scissors') or (user == 'Scissors' and comp == 'Paper') or (user == 'Paper' and comp == 'Rock'):
        return 'User'
    else:
        return 'Computer'

def winner_text(winner):
    if winner == 'User':
        return "good job"
    elif winner == 'Computer':
        return "siuuuuuuu"
    elif winner == 'Draw':
        return "you are copycat"
    else:
        return "what the fuck"

def load_and_resize(path, size=(320,320)):
    img = cv2.imread(path)
    if img is not None:
        img = cv2.resize(img, size)
    return img

IMG_PATHS = {
    'Rock': r"C:\Users\10671\PycharmProjects\pythonProject\relax\visual rock.jpg",
    'Scissors': r"C:\Users\10671\PycharmProjects\pythonProject\relax\visual scissors.jpg",
    'Paper': r"C:\Users\10671\PycharmProjects\pythonProject\relax\visual paper.jpg"
}
comp_images = {}
for k,v in IMG_PATHS.items():
    comp_images[k] = load_and_resize(v)

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Camera open failed.")
    exit()

ret, frame = cap.read()
if not ret:
    print("Frame not read.")
    exit()
frame_h, frame_w = frame.shape[:2]
side_panel_width = 340
side_panel_height = frame_h


bottom_panel_height = 90

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1)
mp_draw = mp.solutions.drawing_utils

countdown_time = 3
show_time = 2.2
game_state = 'waiting'
comp_choice = None
last_result = ''
count_start = 0
result_time = 0
user_gesture = 'what'
last_winner = ''

print("按 空格键 启动倒计时。按 q 键退出。")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(img_rgb)
    hand_sign = None

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            hand_sign = get_hand_sign(hand_landmarks)
    else:
        hand_sign = 'Show hand!'

    # --- 逻辑 ---
    text_for_bottom = ""
    if game_state == 'waiting':
        text_for_bottom = 'Press [Space] to start!'
        comp_choice = None
        key = cv2.waitKey(1)
        if key & 0xFF == ord(' '):
            game_state = 'counting'
            count_start = time.time()
        elif key & 0xFF == ord('q'):
            break
    elif game_state == 'counting':
        elapsed = time.time() - count_start
        count_now = countdown_time - int(elapsed)
        if count_now > 0:
            text_for_bottom = f'Get Ready: {count_now}'
        else:
            if hand_sign in ['Rock','Paper','Scissors']:
                user_gesture = hand_sign
            else:
                user_gesture = '？？？'
            comp_choice = random.choice(['Rock','Paper','Scissors'])
            winner = decide_winner(user_gesture, comp_choice)
            last_winner = winner
            last_result = f'You: {user_gesture}, Computer: {comp_choice}'
            game_state = 'result'
            result_time = time.time()
    elif game_state == 'result':
        if user_gesture == '？？？':
            text_for_bottom = "what the fuck"
        else:
            text_for_bottom = winner_text(last_winner)
        cv2.putText(frame, last_result, (40, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.1, (0, 255, 0), 2)
        if time.time() - result_time > show_time:
            game_state = 'waiting'


    side_panel = np.ones((side_panel_height, side_panel_width, 3), dtype=np.uint8)*245
    if comp_choice and game_state == 'result':
        img = comp_images[comp_choice]
        img_h, img_w = img.shape[:2]
        y_offset = (side_panel_height-img_h)//2
        x_offset = (side_panel_width-img_w)//2
        side_panel[y_offset:y_offset+img_h, x_offset:x_offset+img_w] = img
        cv2.putText(side_panel, f'Computer', (x_offset, y_offset-20), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (50,50,255), 2)
    else:
        cv2.putText(side_panel, 'Waiting...', (60, side_panel_height//2), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (180,180,180), 3)

    # --- 下方大状态栏 ---
    bottom_panel = np.ones((bottom_panel_height, frame_w + side_panel_width, 3), dtype=np.uint8) * 240
    # 让文字始终居中大字号
    if text_for_bottom:
        cv2.putText(bottom_panel, text_for_bottom, ((frame_w+side_panel_width)//2 - 180, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.8, (0,0,255), 4, cv2.LINE_AA)

    # --- 拼接 ---
    main_panel = np.concatenate((frame, side_panel), axis=1)
    total_panel = np.vstack((main_panel, bottom_panel))

    cv2.imshow('Rock Paper Scissors - Left: Camera   Right: Computer', total_panel)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


按 空格键 启动倒计时。按 q 键退出。
